In [52]:
# Inital state probability matrix, given
PI = [0.5,0.5]
# PI = {'Healthy': 0.5, 'Depressed': 0.5}
# state transition probability matrix, given
A = [[0.8,0.2],[0.001,0.999]]
# A = {
#     'Healthy' : {'Healthy': 0.8, 'Depressed': 0.2},
#     'Depressed' : {'Healthy': 0.001, 'Depressed': 0.999},
# }
# Emission or observation probability matrix, given
B = [[0.5,0.1,0.3,0.1,0.0],[0.05,0.35,0.2,0.2,0.2]]
# B = {
#     'Healthy' : {'Movement': 0.5, 'Passive': 0.1, 'Active' : 0.3, 'Texting' : 0.1, 'Access' : 0.0},
#     'Depressed' : {'Movement': 0.05, 'Passive': 0.35, 'Active' : 0.2, 'Texting' : 0.2, 'Access' : 0.2},
# }

# state codes of the observed states, dummy
# Observables = [0,1,2,3,4]
# Observables = ["Movement","Passive","Active","Texting","Access"]
#Observables = ["Movement", "Texting", "Movement", "Texting"]
#states = ["Healthy","Depressed"]
states = [0, 1]
Observables = [0,3,0,3]
#Observables = {"Movement":0, "Passive Social":1, "Active Social":2, "Texting":3, "Access Psych Site":4}

In [53]:
rows = [0] * 3

In [54]:
rows

[0, 0, 0]

In [55]:
matrix = [rows] * 3

In [56]:
matrix

[[0, 0, 0], [0, 0, 0], [0, 0, 0]]

# Filtering

In [57]:
import numpy as np 
from numpy import matrix

def forward(PI, A, B, Observables):
    #Note: alpha_1(i) = pi_i(b_i)(O_1)
    #i.e. alpha_1(i) = PI[i](B[i])(Observables[1])
    #dim of alpha_matrix is n_A x n_Observables
    rows = [0] * len(Observables)
    alpha_matrix = [rows] * len(A)
    print(rows)
    print(len(A))
    print([rows]* len(A))
    for t,t_value in enumerate(Observables):
        print(t)
        for state in states: 
            if t == 0:
                alpha_matrix[state][t] = PI[state] * B[state][t]
                print("Alpha matrix at row {} column {}: {}".format(t, state, alpha_matrix[state][t]))
                print(alpha_matrix)
            else: 
                #alpha_matrix[state][t] = sum(alpha_matrix[l][t-1] * A[l][t-1] for l in states)  * B[state][value]
                for i in range(len(alpha_matrix)):
                    value = 0
                    for l in states:
                        if l == 0:
                            value = alpha_matrix[l][t-1] * A[l][i]  
                            print("Alpha Matrix at row {} column {}: {}".format(l,str(t-1), alpha_matrix[l][t-1]))
                            print("State Probability at row {} column {}: {}".format(l, i, A[l][i]))
                        else:
                            value = value + alpha_matrix[l][t-1]* A[l][i]
                            print("Alpha Matrix at row {} column {}: {}".format(l,str(t-1), alpha_matrix[l][t-1]))
                            print("State Probability at row {} column {}: {}".format(l, i, A[l][i]))
                    print("Observation Probability at row {} column {}: {}".format(state, t_value, B[state][t_value]))
                    value = value * B[state][t_value]
                    alpha_matrix[i][t] = value

            
    print(alpha_matrix)
    alpha_matrix = np.array(alpha_matrix).transpose()


    return alpha_matrix #(normalized forward probability matrix), c (one-dimensional normalization (scaling) coefficient array)

In [92]:
import numpy as np 
from numpy import matrix

def forward(PI, A, B, Observables):
    #rows = [0] * len(Observables)
    alpha_matrix = []
    curr = []
    for t,t_value in enumerate(Observables):
        nex = [0] * len(A)
        for a,a_value in enumerate(A): 
            if t == 0:
                nex[a] = PI[a] * B[a][t_value]
            else:
                for i in range(len(curr)):
                    nex[a] = sum(curr[i]*A[i][l] for l in range(len(A))) * B[a][t_value]
            
        alpha_matrix.append(nex)
        curr = nex
        
    #alpha_matrix = np.transpose(alpha_matrix)

    return alpha_matrix

In [93]:
forward(PI, A, B, Observables)

[[0.25, 0.025],
 [0.0025000000000000005, 0.005000000000000001],
 [0.0025000000000000005, 0.00025000000000000006],
 [2.5000000000000008e-05, 5.0000000000000016e-05]]

In [94]:
def forward_normalized(PI, A, B, Observables):
    alpha = forward(PI, A, B, Observables)
    c = [0] * len(Observables)
    for i,i_value in enumerate(alpha):
        for j,j_value in enumerate(i_value):
            c[j] += i_value[j]
    
    for i in range(len(alpha)):
        for j in range(len(alpha[i])):
            alpha[i][j] = alpha[i][j] * c[j]
    
    return alpha, c

In [95]:
forward_normalized(PI, A, B, Observables)

([[0.06375625, 0.0007575000000000001],
  [0.0006375625000000001, 0.00015150000000000005],
  [0.0006375625000000001, 7.575000000000003e-06],
  [6.375625000000002e-06, 1.5150000000000007e-06]],
 [0.255025, 0.030300000000000004, 0, 0])

# Evaluation

In [96]:
def evaluation_unnormalized(alpha):
    return #probability

In [97]:
def evaluation_normalized(c):
    return probability

# Backward Probability

In [98]:
def backward(PI, A, B, Observables):
    
    beta = []
    b_prev = {}
    for i, value in enumerate(reversed(Observables)):
        b_curr = {}
        for state in states:
            if i == 0:
                b_curr[state] = 1 
            else:
                b_curr[state] = sum(A[state][l] * B[l][value] * b_prev[l] for l in states)

        beta.insert(0,list(b_curr.values()))
        b_prev = b_curr

    #p_beta = sum(PI[l] * B[l][Observables[0]] * b_curr[l] for l in states)
    
    return beta #no of initial states * length of observation sequence

In [99]:
backward(PI, A, B, Observables)

[[0.014028409100000003, 0.0005229484545000001],
 [0.03481800000000001, 0.01012091],
 [0.41000000000000003, 0.05045],
 [1, 1]]

In [100]:
def backward_normalized(PI, A, B, Observables):
    alpha,c = forward_normalized(PI, A, B, Observables)
    beta = backward(PI, A, B, Observables)


    for i in range(len(beta)):
        for j in range(len(beta[i])):
            beta[i][j] = beta[i][j] * c[j]
            
    return beta

In [101]:
backward_normalized(PI, A, B, Observables)

[[0.003577595030727501, 1.5845338171350004e-05],
 [0.008879460450000003, 0.00030666357300000003],
 [0.10456025000000001, 0.0015286350000000002],
 [0.255025, 0.030300000000000004]]

# Smoothing by Forward-Backward Algorithm

In [102]:
def smoothed_probability(alpha, beta, noOfStates, noOfTimeSteps):
    return matrix